# Sentiment Analysis and Consumer Profiling

This script includes cleaning, EDA, feature creation, and some preliminary analysis. 

In [3]:
import pandas as pd
import numpy as np 
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
import re
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib 
import matplotlib.patches as mpatches
import seaborn as sns
from textblob import TextBlob
from collections import Counter
from tqdm import tqdm
import scipy.stats as stats

In [4]:
food = pd.read_csv('./fastfood.csv', dtype=object, index_col=0)

In [5]:
food.shape

(152378, 14)

In [6]:
#dropping malformed data (invalid index)
food.drop(list(food.loc[food['unique_code'] == 'Nobody should be too big to fail...'].index),
          axis=0, inplace=True)

In [7]:
#dropping duplicate entries
food = food.drop_duplicates(subset='unique_code')

In [8]:
#eliminated more than 50% of observations
food.shape

(66504, 14)

In [9]:
food.reset_index(inplace=True, drop=True)

In [10]:
#dropping all rows with all null values
food = food.drop(food[food.isnull().all(axis=1)].index[0], axis=0).reset_index(drop=True)

In [11]:
#Dropping all values that are not company related (only 3 observations)
to_drop = []

for row_num, val in enumerate(food['Company']):
    if val[0] != '@':
        to_drop.append(row_num)
        
food = food.drop(to_drop, axis=0).reset_index(drop=True)

In [12]:
#replacing strings with integers
mapper = {'True': 1, 'False': 0}
food['user_is_verified'] = food.user_is_verified.map(mapper)

In [13]:
#filling nulls and converting data types
food['retweet_count'] = food.retweet_count.astype(int)

In [14]:
#Set to run Midnight and 5pm EST everyday, the times are in UTC, making EST
food['time_tweeted'] = pd.to_datetime(food['time_tweeted']) - pd.Timedelta(hours=4)

In [15]:
food['favorite_count'] = food.favorite_count.astype(int)
food['number_of_people_they_follow'] = food.number_of_people_they_follow.astype(int)
food['number_of_user_tweets'] = food.number_of_user_tweets.astype(int)
food['user_followers_count'] = food.user_followers_count.astype(int)

In [16]:
#creating a basic name category that isnt the handle

mapper = {'@DennysDiner': 'Dennys', '@ChipotleTweets': 'Chipotle',
         '@McDonalds': 'McDonalds', '@Wendys': 'Wendys', '@Starbucks':'Starbucks',
         '@dunkindonuts':'Dunkin_Donuts', '@dominos': 'Dominos', '@shakeshack': 'Shake_Shack',
         '@sonicdrivein': 'Sonic', '@wingstop': 'Wingstop', '@CrackerBarrel': 'Cracker_Barrel', 
         '@redrobinburgers': 'Red_Robin', '@Potbelly': 'Potbelly'}

food['name'] = food.Company.map(mapper)

Examined multiple methods of cleaning for sentiment data, only the final method runs now, but 
have kept my trial methods below (hashed out)

In [17]:
#shouldnt be removing stopwords before sentiment analysis:
#http://www.lrec-conf.org/proceedings/lrec2014/pdf/292_Paper.pdf
#testing different functions for preprocessing text for sentiment analysis


def Text_Cleaner(text, tokens=False):
    """Takes text, eliminates URLS, replaces contractions, tokenizes, 
    removes company names, lower cases, removes calls to twitter handles, 
    returns a string, same as version 1, but only looking at words"""
    text = re.sub(r'(https)[^\s]+', '', text)
    text = re.sub(r'can\'t', 'can not', text, flags=re.IGNORECASE)
    text = re.sub(r'don\'t', 'do not', text, flags=re.IGNORECASE)
    text = re.sub(r'isn\'t', 'is not', text, flags=re.IGNORECASE)
    text = re.sub(r'aren\'t', 'are not', text, flags=re.IGNORECASE)
    text = re.sub(r'wasn\'t', 'was not', text, flags=re.IGNORECASE)
    text = re.sub(r'weren\'t', 'were not', text, flags=re.IGNORECASE)
    text = re.sub(r'haven\'t', 'have not', text, flags=re.IGNORECASE)
    text = re.sub(r'\b(rt|RT)', '', text)
    text = re.sub(r'@[a-zA-Z0-9]+', '', text)
    text = re.sub('#', '', text)
    text = re.sub(r'(wtf)+\b', 'what the fuck', text, flags=re.IGNORECASE)
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)
    if tokens:
        words = [word.lower() for word in words]
        return words
    return ' '.join(words)

# def Text_Cleaner_version_1(text):
#     """Takes text, eliminates URLS, replaces contractions, tokenizes, 
#     removes company names, lower cases, removes calls to twitter handles, 
#     returns a string"""
#     text = re.sub(r'(https)[^\s]+', '', text)
#     text = re.sub(r'\b(rt|RT)', '', text)
#     text = re.sub(r'@[a-zA-Z0-9]+', '', text)
#     text = re.sub('#', '', text)
#     return text

#     LOOKING AT VARIOUS METHODS FOR PREPROCESSING FOR SENTIMENT ANALYSIS
#     tokenizer = RegexpTokenizer(r'\w+')
#     words = tokenizer.tokenize(text)
#     lower = [x.lower() for x in words]
#     words = [word for word in words if word != 'rt']
#     eliminator = [re.sub(r'(mcdon|dunki|denn|redro|sonic|starb|shakesh|domino|crackerb|chipot|wend)[a-z]+','',x)
#                   for x in lower]
#     return ' '.join(eliminator2)

In [18]:
# #Creating a test set of uncleaned data to check the value of the Text_Cleaner functions
# food['text_sentiment_no_clean'] = food['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [19]:
# #converting text with version 1
# food['text_sentiment_v1'] = food['text'].apply(Text_Cleaner_version_1)
# #Calculating sentiment with TextBlob
# food['sentiment_score_v1'] = food['text_sentiment_v1'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [20]:
#converting text with version 2
food['text_sentiment'] = food['text'].apply(Text_Cleaner)
#Calculating sentiment with TextBlob
food['sentiment_score'] = food['text_sentiment'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [21]:
#Looking at overall sentiment by company (version 1) (0 neutral, 1 positive, -1 negative)
food.groupby('Company')['sentiment_score'].mean().sort_values()

Company
@dominos            0.065029
@dunkindonuts       0.094908
@wingstop           0.097293
@sonicdrivein       0.101714
@McDonalds          0.106599
@Wendys             0.108249
@DennysDiner        0.113954
@redrobinburgers    0.121663
@ChipotleTweets     0.147069
@CrackerBarrel      0.153147
@Potbelly           0.162750
@Starbucks          0.173199
@shakeshack         0.184626
Name: sentiment_score, dtype: float64

In [22]:
#creating sentiment dummy variables 

def dummy_maker(val):
    """Takes in a float and returns a dummy based on the value
    to be used in pandas.apply"""
    if val == 0:
        return 0
    elif val > 0:
        return 1
    else:
        return -1

food['sentiment_dummies'] = food['sentiment_score'].apply(dummy_maker)

# food['sentiment_dummies_v2'] = food['sentiment_score_v2'].apply(dummy_maker)

# food['sentiment_dummies_uncleaned'] = food['text_sentiment_no_clean'].apply(dummy_maker)

In [23]:
# print(food['sentiment_dummies_v1'].value_counts()) 
# print(food['sentiment_dummies_v2'].value_counts())
print(food['sentiment_dummies'].value_counts())

 1    28738
 0    27902
-1     9860
Name: sentiment_dummies, dtype: int64


In [24]:
#manually testing reliability of the different measures with a random subset
random_numbers = list(np.random.randint(0, 62415, 5))
for num, val in enumerate(food.iloc[random_numbers, :]['text']):
    print(val)
    print('\n')
    print(food.iloc[[num], :]['sentiment_dummies'].values)

RT @danshiin: @Starbucks @doolsetbangtan @SugasHero RAPLINE IS SHOOK https://t.co/FEzkl1tYKQ


[1]
Hungry to make a difference? In many Mid-Atlantic and Northeastern U.S. states, @chipotletweets is donating 50% of… https://t.co/qd6c82QxJr


[0]
so i’m about to have wingstop for the first time in my life.. don’t let me down @wingstop


[-1]
@sallykuchar @AAAArch @CurbedSF @DennysDiner I thought you'd recognize it.


[0]
RT @dominos: When you just can’t wait to dig in. https://t.co/KMgyrbhiLO


[1]


In [25]:
#function to quickly separate positive/negative tweets by company

def negativity_formatter(company, hourly_rate=False):
    """This function accepts a company and returns either 
    the separate company, positive, negative dataframes (in that order)
    OR the same order plus a dataframe of hourly rates, if hourly_rate = True"""
    df = food.loc[food['name'] == company]
    positive_df = df.loc[df['sentiment_dummies'] == 1]
    negative_df = df.loc[df['sentiment_dummies'] == -1]
    if not hourly_rate:
        return df, positive_df, negative_df
    else:
        rate_df = pd.DataFrame()
        rate_df['pos_count'] = positive_df.groupby(positive_df['time_tweeted'].dt.hour)['Company'].count()
        rate_df['neg_count'] = negative_df.groupby(negative_df['time_tweeted'].dt.hour)['Company'].count()
        rate_df['rate'] = rate_df['neg_count'] / (rate_df['pos_count'] + rate_df['neg_count'])
        return rate_df
        

In [26]:
#43% of tweets are retweets
food.loc[food['text'].str[:2] == 'RT'].shape[0] / food.shape[0]

0.43109774436090226

In [27]:
#Creating a dummy for whether the tweet is a retweet or not
retweets = []
for val in food['text']:
    if val[:2] == 'RT':
        retweets.append(1)
    else: 
        retweets.append(0)
food['is_a_retweet'] = retweets

In [28]:
#getting rid of tweets without a user 
food = food.drop(list(food.loc[(food['user_name'].isnull())].index), axis=0)

In [29]:
#dropping regional affiliates
associated_comps = []
names = []
for num, name in enumerate(food['user_name']):
    test = re.findall(r'\A(mcdon|dunki|redro|starbuc|shakesh|domino|crackerb)[a-z]+'
                      , name, flags=re.IGNORECASE)
    if test:
        if name not in ['Dunkin Fails', 'Dunkin Kitti', "McDonald's employee"]:
            associated_comps.append(num)
            names.append(name)
            
food = food.drop(associated_comps, axis=0)

In [30]:
#favorite count is heavilly skewed, but keep for now
food['favorite_count'].describe()

count    66441.000000
mean         0.508662
std          5.554575
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        857.000000
Name: favorite_count, dtype: float64

## Beginning to look at user profiles by company

In [31]:
#customer profiles still skewed by large means- looking at medians & means
#creating a subset to examine the numeric characteristics of each company's customers
individual_users = food.drop_duplicates(subset='user_name')

customer_numeric_df = individual_users.groupby('name').agg({'favorite_count': ['mean', 'max'], 
                                      'number_of_people_they_follow': ['median', 'mean'],
                                      'number_of_user_tweets': ['median', 'mean'],
                                      'retweet_count': ['median', 'mean'],
                                      'user_followers_count': ['median', 'mean'],
                                       'user_is_verified': ['mean'],
                                      'sentiment_score': ['mean'], 
                                      'is_a_retweet': 'mean',
                                      'Company': 'count'})

customer_numeric_df.columns = [' '.join(col).strip() for col in customer_numeric_df.columns.values]

In [32]:
#Not enough observations for Potbelly (684), RedRobin (1463), dropping
food = food.drop(list(food.loc[(food['Company']== '@redrobinburgers')|
                   (food['Company']== '@Potbelly')].index), axis=0)
food = food.reset_index(drop=True)

In [33]:
#MISSING VALUES FOR CERTAIN COMPANIES AT CERTAIN HOURS- hourly is flawed
# negativity_by_comp = pd.DataFrame()
# for val in food.name.unique().tolist():
#     if val != 'McDonalds':
#         values = list(negativity_formatter(val, hourly_rate=True)['rate'].values)
#         negativity_by_comp[val] = values

In [34]:
food['day_date'] = food['time_tweeted'].dt.day
food['weekday'] = food['time_tweeted'].dt.weekday

## Beginning look at stock movements and sentiment

In [35]:
#specifying trading days with sufficient information range
stock_analysis = food.loc[(food['day_date'] > 11) & (food['day_date'] < 31)]

In [36]:
stock_analysis.shape

(63463, 21)

In [37]:
stocks = pd.read_csv('./twitter_stocks.csv')

In [38]:
stocks.drop(['High', 'Low', 'Adj Close'], axis=1, inplace=True)

In [39]:
stocks['change'] = stocks['Close'] - stocks['Open']

In [40]:
#creating datetime, and locating common trading dates
stocks['Date'] = pd.to_datetime(stocks.Date)
stocks['day'] = stocks['Date'].dt.day

In [41]:
stocks['day'] = stocks['Date'].dt.day

In [42]:
stocks = stocks.loc[(stocks['day'] > 11) & (stocks['day'] < 31)]

In [43]:
#making sure values match before merging
stocks['Name'] = stocks.Name.str.replace('Shack Shack', 'Shake_Shack')
stocks['Name'] = stocks.Name.str.replace('Cracker Barrel', 'Cracker_Barrel')
stocks['Name'] = stocks.Name.str.replace('Dunkin Donuts', 'Dunkin_Donuts')

In [44]:
#grouping sentiments to merge
stock_analysis = stock_analysis.groupby(['day_date', 'name'], as_index=False).agg(
                                                    {'sentiment_score':'mean',
                                                       'Company':'count'})

stock_analysis.columns =  ['day_date', 'name', 'sentiment_score', 'num_observations']

In [45]:
merged_stock = pd.merge(stock_analysis, stocks, left_on=['day_date', 'name'], right_on=['day', 'Name'])

In [46]:
#dropping redundant columns 
merged_stock.drop(['day', 'Name'], axis=1, inplace=True)

In [47]:
merged_stock = merged_stock.sort_values(['name', 'day_date'])

In [48]:
#correlations between sentiment_score and various other indicators (volume/change/market_cap)
merged_stock.corr()['sentiment_score']

day_date           -0.094365
sentiment_score     1.000000
num_observations   -0.013644
Open               -0.014959
Close              -0.012543
Volume              0.145016
Market Cap          0.089305
change              0.102586
Name: sentiment_score, dtype: float64

## Looking at Profiles of negative/positive sentiments

In [49]:
#set of english vocabulary
english_vocab = set(w.lower() for w in nltk.corpus.words.words())

In [50]:
def lang_composition(company):
    """Takes a company name and returns the total number of words present, 
    the number of unique words present, and the number of those words that are in the English
    language"""
    all_tweets, positive, negative = negativity_formatter(company)
    all_text = ' '
    pos_text = ' '
    neg_text = ' '
    for val in all_tweets.text:
        if val[:2] != 'RT':
            all_text = all_text + ' ' + val
    for val in positive.text:
        if val[:2] != 'RT':
            pos_text = pos_text + ' ' + val
    for val in negative.text:
        if val[:2] != 'RT':
            neg_text = neg_text + ' ' + val
    all_words = Text_Cleaner(all_text, tokens = True)
    pos_words = Text_Cleaner(pos_text, tokens = True)
    neg_words = Text_Cleaner(neg_text, tokens = True)
    info = [company]
    info.extend([len(all_words), len(pos_words), len(neg_words)])
    info.extend([len(set(all_words)), len(set(pos_words)), len(set(neg_words))])
    all_clean_words = [word for word in set(all_words) if word in english_vocab]
    pos_clean_words = [word for word in set(pos_words) if word in english_vocab]
    neg_clean_words = [word for word in set(neg_words) if word in english_vocab]
    info.extend([len(all_clean_words), len(pos_clean_words), len(neg_clean_words)])
    return info

In [51]:
#getting language composition for all companies
all_comps = []
for name in list(food.name.unique()):
    all_comps.append(lang_composition(name))
    
lang_comp_df = pd.DataFrame(all_comps)

In [52]:
#renaming all columns
lang_comp_df.columns = ['name', 'all_words','pos_all_words', 'neg_all_words', 'unique_words', 'pos_unique_words', 'neg_unique_words', 
                        'english_words', 'pos_english_words', 'neg_english_words']

In [53]:
#creating percentages for analysis
lang_comp_df['percent_unique'] = lang_comp_df['unique_words'] / lang_comp_df['all_words'] * 100
lang_comp_df['percent_english'] = lang_comp_df['english_words'] / lang_comp_df['unique_words'] * 100
lang_comp_df['pos_percent_unique'] = lang_comp_df['pos_unique_words'] / lang_comp_df['pos_all_words'] * 100
lang_comp_df['pos_percent_english'] = lang_comp_df['pos_english_words'] / lang_comp_df['pos_unique_words'] * 100
lang_comp_df['neg_percent_unique'] = lang_comp_df['neg_unique_words'] / lang_comp_df['neg_all_words'] * 100
lang_comp_df['neg_percent_english'] = lang_comp_df['neg_english_words'] / lang_comp_df['neg_unique_words'] * 100

In [54]:
#Merging sentiments 
lang_comp_df = pd.merge(lang_comp_df, food.groupby('name', as_index=False)['sentiment_score'].mean(), 
       on='name')

#Merging only negative sentiments 
food_neg = food.loc[food['sentiment_dummies'] == -1].groupby('name',as_index=False)['sentiment_score'].mean()
food_neg.columns = ['name', 'neg_sentiment']
lang_comp_df = pd.merge(lang_comp_df, food_neg, on='name')

#Merging only positive sentiments
food_pos = food.loc[food['sentiment_dummies'] == 1].groupby('name',as_index=False)['sentiment_score'].mean()
food_neg.columns = ['name', 'pos_sentiment']
lang_comp_df = pd.merge(lang_comp_df, food_pos, on='name')

In [55]:
#strongest correlations with overall sentiment- percent unique (.31), percent English (0.29)
lang_comp_df.corr()['sentiment_score_x']

all_words             -0.453531
pos_all_words         -0.344574
neg_all_words         -0.553493
unique_words          -0.280634
pos_unique_words      -0.199107
neg_unique_words      -0.434417
english_words         -0.243947
pos_english_words     -0.180855
neg_english_words     -0.383851
percent_unique         0.311866
percent_english        0.298892
pos_percent_unique     0.193846
pos_percent_english    0.153271
neg_percent_unique     0.532893
neg_percent_english    0.581626
sentiment_score_x      1.000000
neg_sentiment          0.536657
sentiment_score_y      0.512859
Name: sentiment_score_x, dtype: float64

## Looking at company tweets

In [56]:
company_tweets = pd.read_csv('./company_tweets.csv', index_col=0)

In [57]:
company_tweets.shape

(10920, 11)

In [58]:
company_tweets['time_tweeted'] = pd.to_datetime(company_tweets['time_tweeted'])

In [59]:
#there are no duplicates
company_tweets.unique_code.drop_duplicates(inplace=True)

In [60]:
#dropping potbelly & redrobin
company_tweets.drop(list(company_tweets.loc[(company_tweets['name'] == '@redrobinburgers')
                  |(company_tweets['name'] == '@Potbelly')].index), axis=0,
                   inplace=True)

In [61]:
#getting normally spelled names
mapper = {'@DennysDiner': 'Dennys', '@ChipotleTweets': 'Chipotle',
         '@McDonalds': 'McDonalds', '@Wendys': 'Wendys', '@Starbucks':'Starbucks',
         '@dunkindonuts':'Dunkin_Donuts', '@dominos': 'Dominos', '@shakeshack': 'Shake_Shack',
         '@sonicdrivein': 'Sonic', '@wingstop': 'Wingstop', '@CrackerBarrel': 'Cracker_Barrel', 
         '@redrobinburgers': 'Red_Robin', '@Potbelly': 'Potbelly'}

company_tweets['Company'] = company_tweets.name.map(mapper)

In [62]:
food.time_tweeted.max()

Timestamp('2018-03-30 09:48:49')

In [63]:
company_tweets.reset_index(drop=True, inplace=True)

In [64]:
#dropping all company tweets that occurred after consumer tweet collection stopped
company_tweets.drop(list(
    company_tweets.loc[company_tweets['time_tweeted'] > food.time_tweeted.max()].index), axis=0, inplace=True)

In [65]:
print(company_tweets.shape)
company_tweets = company_tweets.reset_index(drop=True)

(8080, 12)


In [66]:
#dropping values that are before consumer observations
company_tweets.drop(list(company_tweets.loc[company_tweets['time_tweeted'] < 
                                       food.time_tweeted.min()].index), inplace=True)

In [67]:
company_tweets = company_tweets.reset_index(drop=True)

In [68]:
#Getting the hashtags for each company
hashtags = {}
callnames = {}
for company in tqdm(list(company_tweets.Company.unique())): 
    df = company_tweets.loc[company_tweets['Company'] == company]
    hash_base = []
    call_base = []
    for val in df['text']:
        hashes = re.findall(r'#[A-Za-z0-9]+\b', val)
        calls = re.findall(r'@[A-Za-z0-9]+\b', val)
        hash_base.extend(hashes)
        call_base.extend(calls)
    hashtags[company] = hash_base
    callnames[company] = call_base

100%|██████████| 11/11 [00:00<00:00, 240.76it/s]


In [69]:
tokenizer = RegexpTokenizer(r'#[A-Za-z0-9]+\b')
#Only interested in customers independently using the hashtags
hash_check = food.loc[food['is_a_retweet'] == 0]
hash_check['text'] = hash_check.text.apply(lambda x: tokenizer.tokenize(x))

/Users/slevin886/anaconda2/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [70]:
#Identifying customer use of company hashtags

company_counts = {}
for company in list(company_tweets.Company.unique()):
    hash_comp = hash_check.loc[hash_check['name'] == company]
    hashes = {}
    for val in hash_comp['text']:
        for word in val:
            if word in hashtags[company]:
                try: 
                    hashes[word] += 1
                except:
                    hashes[word] = 1
    company_counts[company] = hashes

In [71]:
#putting statistics on hashtags/use into new company profile dataframe
new_df = pd.DataFrame(company_counts).T.sum(axis=1).reset_index()
new_df.columns = ['name', 'customer_hash_use']
new_df['customer_unique_hashes'] = pd.DataFrame(company_counts).T.count(axis=1).values

In [72]:
#examining company specific twitter behavior
comp_hash_uses = []
comp_unique_hashes = []
comp_handle_uses = []
comp_unique_handle_uses = []
for company in new_df.name.tolist():
    comp_hash_uses.append(len(hashtags[company]))
    comp_unique_hashes.append(len(set(hashtags[company])))
    comp_handle_uses.append(len(callnames[company]))
    comp_unique_handle_uses.append(len(set(callnames[company])))

new_df['comp_hash_uses'] = comp_hash_uses
new_df['comp_unique_hashes'] = comp_unique_hashes
new_df['comp_handle_uses'] = comp_handle_uses
new_df['comp_unique_handle_uses'] = comp_unique_handle_uses

In [73]:
#grouping company tweet info
comp_grouped = company_tweets.groupby('Company', as_index=False).mean().drop(
                            ['is_a_retweet', 'is_quote_status', 'unique_code'], axis=1)

In [74]:
comp_merged = pd.merge(comp_grouped, new_df, left_on='Company', right_on='name')
comp_merged.drop('name', axis=1, inplace=True)

In [75]:
comp_merged['market_cap'] = merged_stock.groupby('name')['Market Cap'].mean().values

In [76]:
comp_merged['sentiment'] = food.groupby('name')['sentiment_score'].mean().values

In [77]:
print('Sentiment Correlations')
comp_merged.corr()['sentiment'].sort_values()[:-1]

Sentiment Correlations


comp_unique_handle_uses   -0.154493
retweet_count             -0.072436
number_of_tweets_total    -0.070891
comp_hash_uses            -0.041042
customer_unique_hashes     0.046609
comp_unique_hashes         0.091145
market_cap                 0.110416
comp_handle_uses           0.251164
customer_hash_use          0.303470
favorite_count             0.311198
company_followers_count    0.333409
followers_count            0.333409
Name: sentiment, dtype: float64

## Day of week optimizer

In [78]:
#ensuring that every weekday has equal representation (Friday will be slightly short-changed (by ~3 hours)
day_week = food.loc[(food['time_tweeted'] > '2018-03-09')]

In [79]:
#converting day nums to names, grouping by company and day of week and aggregating. 
mapper = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
day_week = day_week.groupby(['name', 'weekday'], as_index=False).agg({'sentiment_score':'mean', 'user_name': 'count',
                                                          'retweet_count':'mean', 'favorite_count':'mean'})

day_week['weekday'] = day_week['weekday'].map(mapper)

In [80]:
#including average sentiment for visualization
merge = day_week.groupby('weekday', as_index=False)['sentiment_score'].mean()
merge.columns = ['weekday', 'avg_sentiment']
day_week = pd.merge(day_week, merge, on='weekday')

In [81]:
print('number of tweets by day')
day_week.groupby('weekday')['user_name'].sum().sort_values()

number of tweets by day


weekday
Sunday        4345
Saturday      7259
Monday        7549
Wednesday     9024
Friday        9840
Thursday     12736
Tuesday      13541
Name: user_name, dtype: int64

In [82]:
print('average sentiment by day')
day_week.groupby('weekday')['sentiment_score'].mean().sort_values()

average sentiment by day


weekday
Saturday     0.089578
Sunday       0.100064
Friday       0.116223
Thursday     0.116878
Monday       0.125332
Wednesday    0.131134
Tuesday      0.156336
Name: sentiment_score, dtype: float64

## Beginning profile analysis

In [83]:
profiles = food.dropna(subset=['user_profile_text']).reset_index(drop=True)

In [84]:
profiles.shape

(54646, 21)

In [85]:
#only interested in unique profiles
profiles = profiles.drop_duplicates(subset='user_name').reset_index(drop=True)

In [86]:
profiles.shape

(40482, 21)

In [87]:
#dropping columns I don't need for this
profiles.drop(['user_coordinates', 'unique_code', 'day_date', 'text_sentiment'], axis=1, inplace=True)

In [88]:
#adding an hour dummy
profiles['hour'] = profiles.time_tweeted.dt.hour

In [89]:
#getting rid of columns I don't need anymore
profiles = profiles.drop(['time_tweeted', 'Company'], axis=1)

In [90]:
#Maybe the number of hashtags a person uses is indicative of their personality/disposition towards the product
def num_hashes(text):
    """Counts the number of hashtags in the profile"""
    text = text.split()
    num_hashes = 0
    for word in text:
        if word[0] == '#':
            num_hashes = num_hashes + 1
    return num_hashes

profiles['number_of_hashes'] = profiles['user_profile_text'].apply(num_hashes)

In [91]:
#8% of users use a hashtag in their profile
np.sum(profiles.number_of_hashes > 1) / profiles.shape[0]

0.08248110271231658

In [92]:
#adding a dummy variable if the user lists a location, possibly indicator
profiles['lists_location'] = profiles.user_location.isnull().astype(int)

#unreliable for now to include
profiles = profiles.drop('user_location', axis=1)

In [93]:
def cleaning_additional(text):
    """standardizing contractions, removing urls, removing retweet indicators, removing handles"""
    text = re.sub(r'https[\S]+', ' ', text)
    text = re.sub(r'can\'t', 'can not', text, flags=re.IGNORECASE)
    text = re.sub(r'don\'t', 'do not', text, flags=re.IGNORECASE)
    text = re.sub(r'isn\'t', 'is not', text, flags=re.IGNORECASE)
    text = re.sub(r'aren\'t', 'are not', text, flags=re.IGNORECASE)
    text = re.sub(r'wasn\'t', 'was not', text, flags=re.IGNORECASE)
    text = re.sub(r'weren\'t', 'were not', text, flags=re.IGNORECASE)
    text = re.sub(r'haven\'t', 'have not', text, flags=re.IGNORECASE)
    text = re.sub(r'\b(rt|RT)', ' ', text)
    text = re.sub(r'@[\S]+', ' ', text)
    text = re.sub(r'[0-9]*[a-zA-Z]+[0-9]+[a-zA-Z]*[0-9]*[a-zA-Z]*', ' ', text)
    return text

profiles['cleaned'] = profiles.user_profile_text.apply(cleaning_additional)

In [94]:
def punctuation_cleaner(text):
    """Removes punctuation and places spaces"""
    text = re.sub(r'\\n', ' ', text)      
    text = re.sub(r'[!|?|.|,|(|)|||[|]|/|\\|-]', ' ', text)
    return text

profiles['cleaned'] = profiles.cleaned.apply(punctuation_cleaner)

In [95]:
def split_hashes(text):
    """Turns camel case hashtags into separate readable words"""
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)
    split2 = []
    for val in words:
        if val[0] == '#':
            words = ' '.join(re.findall('[A-Z][^A-Z]*', val))
            split2.append(words)
        else:
            split2.append(val)
    return ' '.join(split2)

profiles['without_hashes'] = profiles.cleaned.apply(split_hashes)

### T-test for mean difference between sentiment with/without retweets

In [96]:
full = [0.065029, 0.094908, 0.097293, 0.101714, 0.106599, 0.108249, 0.113954, 0.121663, 0.147069, 0.153147, 0.162750, 0.173199, 0.184626]
no_retweet = [0.089,0.132,0.072,0.0439,0.089,0.055887,0.124,0.089954,0.111304,0.134573,0.084565]

stats.ttest_ind(full, no_retweet)

Ttest_indResult(statistic=2.3715728048141727, pvalue=0.026886738879467794)

## Looking at possible occupational/regional tendencies with a noun phrase function. 

In [97]:
#words that aren't caught that don't add value to function below
drop_words = ['love','don t', 'twitter', 'don', 'tweets', 'opinions', 'your',
             'born', 'never']

In [45]:
def Profile_Noun_Finder(text):
    """Takes text, eliminates non-alpha characters, and returns all nouns"""
    text = re.sub(r'(https)[^\s]+', '', text)
    text = re.sub(r'[^a-zA-Z]+', ' ', text)
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)
    lowered = [word.lower() for word in words]
    words = [word for word in lowered if len(word) > 2]
    words = [word for word in words if word not in drop_words]
    new_text = ' '.join(words)
    wiki = TextBlob(new_text)
    return list(wiki.noun_phrases)